In [3]:
import pandas as pd
pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', None)

import datetime as datetime
import altair as alt

In [4]:
df = pd.read_csv('../data/data_restrictions.csv', delimiter=',', encoding ='latin-1')
geo = pd.read_csv('../data/data_restrictions_geo.csv', delimiter=',', encoding ='latin-1')

c:\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (23,25,26,27,28,30,31,32,33,34,35,37,38,39,48,51,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.loc[:,['unit', 'date_start', 'date_end', 'country', 'region', 'entry_type', 'update_type', 'target_country', 'type']].head()

,unit,date_start,date_end,country,region,entry_type,update_type,target_country,type
0,1,2020-03-06,2022-02-10,Afghanistan,Southern Asia,new_entry,NaN,Afghanistan,Anti-Disinformation Measures
1,1,2020-03-16,2020-03-16,Afghanistan,Southern Asia,new_entry,NaN,Afghanistan,Anti-Disinformation Measures
2,1,2020-05-14,2020-05-15,Afghanistan,Southern Asia,new_entry,NaN,Afghanistan,Anti-Disinformation Measures
3,1,2020-05-28,NaN,Afghanistan,Southern Asia,new_entry,NaN,Afghanistan,Anti-Disinformation Measures
4,1,2020-05-28,2020-05-29,Afghanistan,Southern Asia,new_entry,NaN,Afghanistan,Anti-Disinformation Measures


In [6]:
geo.loc[:,['continent', 'region', 'country']].head()

,continent,region,country
0,Asia,Southern Asia,Afghanistan
1,Europe,Southern Europe,Albania
2,Americas,Northern America,Antarctica
3,Africa,Northern Africa,Algeria
4,Oceania,Polynesia,American Samoa


In [7]:
df['parsed_start_date'] = pd.to_datetime(df['date_start'], format='%Y-%m-%d')
df['parsed_end_date'] = pd.to_datetime(df['date_end'], format='%Y-%m-%d')

df.loc[:,['parsed_start_date', 'parsed_end_date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104852 entries, 0 to 104851
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   parsed_start_date  104852 non-null  datetime64[ns]
 1   parsed_end_date    61151 non-null   datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.6 MB


In [15]:
end_policy = 'End of Policy' 
restriction_border_inner = 'Internal Border Restrictions' 
restriction_border_outer = 'External Border Restrictions'

# exclude end of restriction events and filter for border restrictions only
df_cleaned = df[(df['update_type']!=end_policy) & ((df['type']==restriction_border_inner) | (df['type']==restriction_border_outer))]  

In [17]:
df_cleaned.loc[:,['unit', 'date_start', 'date_end', 'country', 'region', 'entry_type', 'update_type', 'target_country', 'type']].head()

,unit,date_start,date_end,country,region,entry_type,update_type,target_country,type
117,1,2020-01-27,NaN,Afghanistan,Southern Asia,new_entry,NaN,China,External Border Restrictions
118,1,2020-02-03,NaN,Afghanistan,Southern Asia,new_entry,NaN,China,External Border Restrictions
119,1,2020-02-12,NaN,Afghanistan,Southern Asia,new_entry,NaN,NaN,External Border Restrictions
120,1,2020-02-23,2020-02-24,Afghanistan,Southern Asia,new_entry,NaN,Iran,External Border Restrictions
121,1,2020-02-23,NaN,Afghanistan,Southern Asia,new_entry,NaN,Iran,External Border Restrictions


In [18]:
earliest_start = df['parsed_start_date'].min()
latest_end = datetime.date(2022, 2, 10)

In [19]:
delta = datetime.timedelta(days=1)

In [52]:
def region_analyse(region):
    inspect_date = earliest_start # start with the earliest possible date
    list_of_restrictions_by_date=[]

    while(inspect_date <= latest_end): # for as long as the the date to inspect is smaller or equal the latest end date
        date_dict = {} # create an empty dictionary to later store retrieved values
        date_dict['date']=inspect_date # save inspected date into said dictionary
        # filter df for entries smaller start dates or larger end dates:
        df_temp = df_cleaned[(df_cleaned['parsed_start_date']<=inspect_date) & (df_cleaned['parsed_end_date']>=inspect_date) & df_cleaned['region']==region]  
        date_dict['restrictions_on_date'] = len(df_temp) # length of filtered df equals number of restrictions on that date

        list_of_restrictions_by_date.append(date_dict) # add date dict to list

        inspect_date = inspect_date + delta # put date counter one day up

    df_restrictions_by_date = pd.DataFrame(list_of_restrictions_by_date)  # convert list to dataframe
    
    region_chart = alt.Chart(df_restrictions_by_date).mark_line().encode(
        x='date',
        y='restrictions_on_date'
    )
        
    return region_chart # show dataframe
    return df_restrictions_by_date

In [53]:
region_analyse('Southern Asia') 

# The condition is still not working

alt.Chart(...)